앞서 살펴본 방법들을 이용하여 다양한 군집화 알고리즘의 사용법을 연습합니다.

In [1]:
from bokeh.plotting import output_notebook, show
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd


output_notebook()

iris = load_iris()
print(iris.keys())

# ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
columns = [name[:-5].replace(' ', '_') for name in iris['feature_names']]

iris_df = pd.DataFrame(data=iris.data, columns=columns)
iris_df['target'] = iris.target

iris_df.head(5)

Loading BokehJS ...

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


아래의 내용은 이전에 살펴보았던 내용들이기 중복적인 설명은 생략합니다.

In [2]:
from bokeh.palettes import Set1
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

marker_list = 'circle square triangle asterisk circle_x square_x '\
    'inverted_triangle x circle_cross square_cross diamond cross'.split()

def scatterplot_bokeh(iris_df, labels=None, title=None, height=400, width=400,
    x='petal_length', y='petal_width', alpha=1.0, show_inline=True):

    # prepare x, y, labels
    if labels is None:
        labels = iris_df['target']
    if isinstance(x, str):
        x = iris_df[x]
        y = iris_df[y]
    if isinstance(alpha, float) or isinstance(alpha, int):
        alpha = [alpha] * len(iris_df)

    data = iris_df.copy()
    data['label'] = labels
    data['x'] = x
    data['y'] = y

    # set color & marker
    n_labels = np.unique(labels).shape[0]
    palette = Set1[max(3, n_labels)]
    data['color'] = [palette[l] if l >= 0 else 'lightgrey' for l in labels]
    data['marker'] = [marker_list[t] for t in iris_df['target']]
    data['alpha'] = alpha

    # as ColumnDataSource
    source = ColumnDataSource(data)

    # prepare hover tool
    tooltips = [
        ('index', '$index'),
        ('(sepal length, sepal width)', '(@sepal_length, @sepal_width)'),
        ('(petal length, petal width)', '(@petal_length, @petal_width)'),
        ('cluster label', '@label'),
        ('true label', '@target')
    ]

    # draw figure
    p = figure(title=title, height=height, width=width, tooltips=tooltips)
    # p.background_fill_color='#202020'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.scatter(x='x', y='y', size=7, line_color='white', alpha='alpha',
        color='color', marker='marker', source=source, legend_field='label')

    if show_inline:
        show(p)
    return p

IRIS data 는 4 차원 데이터이며, 군집화 결과를 시각적으로 확인하기 위해서 데이터를 2 차원으로 압축해야 합니다. PCA 를 이용하여 4 차원의 데이터를 2 차원으로 변환합니다. 이때는 군집화 결과가 전혀 이용되지 않습니다. 아래 그림의 좌측은 iris data 의 두 개의 column 을 이용한 결과이며, 우측은 PCA 를 이용한 결과입니다. 사실 IRIS data 는 클래스 분류에 큰 어려움이 없는 데이터이기 때문에 두 개의 features 를 이용한 결과나 PCA 를 이용한 결과나 비슷하지만, 이러한 방법은 더 큰 고차원 데이터의 레이블 정보를 시각화 하는데 매우 유용합니다.

In [3]:
from bokeh.layouts import gridplot
from sklearn.decomposition import PCA

z = PCA(n_components=2).fit_transform(iris.data)
zx, zy = z[:,0], z[:,1]

gp_iris = gridplot([[
    scatterplot_bokeh(iris_df, title='IRIS data', show_inline=False),
    scatterplot_bokeh(iris_df, x=zx, y=zy, title='IRIS data (PCA)', show_inline=False),
]])

show(gp_iris)

아래는 다양한 군집화 알고리즘에 대하여 중요한 hyper parameters 를 변경하며 학습한 결과입니다.

## DBSCAN

In [4]:
from sklearn.cluster import DBSCAN

eps_array = [0.5, 0.6, 0.7]
min_samples_array = [10, 15, 20]

grid_dbscan = []
for eps in eps_array:
    grid_row = []
    for min_samples in min_samples_array:

        # training DBSCAN
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
        labels = dbscan.fit_predict(iris.data)

        # prepare scatterplot
        title = f'DBSCAN eps={eps}, min_samples={min_samples}'
        p = scatterplot_bokeh(iris_df, labels=labels, x=zx, y=zy,
            title=title, height=350, width=350, show_inline=False)

        grid_row.append(p)
    grid_dbscan.append(grid_row)

gp_dbscan = gridplot(grid_dbscan)
show(gp_dbscan)

## Gaussian Mixture Model

In [5]:
from sklearn.mixture import GaussianMixture

grid_gmm = []

# training GMM
gmm = GaussianMixture(n_components=3, random_state=1234)
labels = gmm.fit_predict(iris.data)

# prepare scatterplot
p = scatterplot_bokeh(iris_df, labels=labels, x=zx, y=zy,
    title='GMM', height=400, width=400, show_inline=False)
grid_gmm.append(p)

# # draw scatterplot with probability
proba = gmm.predict_proba(iris.data)
proba.sort(axis=1)
proba = proba[:,-1]
labels[np.where(proba <= 0.8)] = -1

alpha = proba ** 5
p = scatterplot_bokeh(iris_df, labels=labels, x=zx, y=zy,
    title='GMM', height=400, width=400, alpha=alpha, show_inline=False)
grid_gmm.append(p)

gp_gmm = gridplot([grid_gmm])
show(gp_gmm)

## k-means

In [6]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

grid_kmeans = []
for k in [2, 3, 5, 8]:

    # training k-means
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=30, verbose=0)
    labels = kmeans.fit_predict(iris.data)

    # Silhouette score
    sil_score = silhouette_score(iris.data, labels)

    # prepare scatterplot
    title = f'k-means k={k}, Silhouette={sil_score:.4}'
    p = scatterplot_bokeh(iris_df, labels=labels, x=zx, y=zy,
        title=title, height=350, width=350, show_inline=False)
    grid_kmeans.append(p)

grid_kmeans = [
    [grid_kmeans[0], grid_kmeans[1]],
    [grid_kmeans[2], grid_kmeans[3]]
]

gp_kmeans = gridplot(grid_kmeans)
show(gp_kmeans)

## Hierarchical clustering

In [7]:
from sklearn.cluster import AgglomerativeClustering

grid_hc = []
k_array = [2, 3, 5]
threshold_array = [5.0, 4.0, 3.0]

row = []
for k in k_array:    
    # training hierarchical clustering
    hc = AgglomerativeClustering(n_clusters=k,
        linkage='ward', affinity='euclidean')
    labels = hc.fit_predict(iris.data)

    # prepare scatterplot
    title = f'HC k={k}'
    p = scatterplot_bokeh(iris_df, labels=labels, x=zx, y=zy,
        title=title, height=350, width=350, show_inline=False)
    row.append(p)
grid_hc.append(row)

row = []
for threshold in threshold_array:
    # training hierarchical clustering
    hc = AgglomerativeClustering(n_clusters=None, linkage='ward',
        affinity='euclidean', distance_threshold=threshold)
    labels = hc.fit_predict(iris.data)

    # prepare scatterplot
    n_clusters = np.unique(labels).shape[0]
    title = f'HC threshold={threshold}, n clusters={n_clusters}'
    p = scatterplot_bokeh(iris_df, labels=labels, x=zx, y=zy,
        title=title, height=350, width=350, show_inline=False)
    row.append(p)
grid_hc.append(row)

gp_hc = gridplot(grid_hc)
show(gp_hc)

## Plotting with UMAP

그러나 앞서 언급한 것처럼 PCA 를 학습하는 과정에는 군집화 학습 결과가 전혀 이용되지 않습니다. 우리는 UMAP 이 제공하는 supervised embedding 을 이용하여 군집별로 점들이 떨어진 형태로 plotting 이 되도록 유도해 봅니다.

Hierarchical clustering algorith 을 학습합니다. 군집의 개수는 3 개로 설정합니다.

In [8]:
hc = AgglomerativeClustering(n_clusters=3,
    linkage='ward', affinity='euclidean')
labels = hc.fit_predict(iris.data)

UMAP 을 이용하여 supervised embedding 을 학습합니다. `fit(X=data, y=labels)` 처럼 `y` 를 입력합니다.

In [11]:
import umap
import warnings
warnings.filterwarnings('ignore')

embedding = umap.UMAP(n_neighbors=10).fit_transform(X=iris.data, y=labels)

군집화 결과를 살펴봅니다. 의도한것처럼 군집 별로 점들이 멀리 떨어져 표현됩니다. 하지만 자세히 살펴보면 군집화 결과가 좋지 않기 때문에 세모와 네모의 모양이 녹색과 빨간색에 겹쳐서 표현된 것을 확인할 수 있습니다. 이는 적절한 군집화 알고리즘을 학습하여 해결해야 합니다.

In [12]:
p0 = scatterplot_bokeh(iris_df, labels=labels, x=embedding[:,0], y=embedding[:,1],
    title="HC(k=3) + UMAP", height=350, width=350, show_inline=False)
p1 = scatterplot_bokeh(iris_df, labels=labels, x=zx, y=zy,
    title="HC(k=3) + PCA", height=350, width=350, show_inline=False)
gp_umap = gridplot([[p0, p1]])
show(gp_umap)